Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [13]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
  # Set final layer variables
  weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden1, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction1, weights) + biases)

  #test operation
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)


In [14]:
num_steps = 50001 

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 820.602417
Minibatch accuracy: 11.7%
Validation accuracy: 29.9%
Minibatch loss at step 500: 227.993927
Minibatch accuracy: 79.7%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 104.004990
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 48.548004
Minibatch accuracy: 91.4%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 22.993826
Minibatch accuracy: 91.4%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 11.160647
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 5.583574
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 3500: 2.981484
Minibatch accuracy: 86.7%
Validation accuracy: 87.4%
Minibatch loss at step 4000: 1.606094
Minibatch accuracy: 89.8%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 0.995699
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 5000: 0.782194
Minibatch accuracy: 89.1%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---


In [7]:
batch_size = 5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  loss = loss + 0.5*tf.nn.l2_loss(tf.nn.l2_normalize(weights, dim=0))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [24]:
num_steps = 10001 

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 11.814745
Minibatch accuracy: 0.0%
Validation accuracy: 12.4%
Minibatch loss at step 500: 9.452267
Minibatch accuracy: 80.0%
Validation accuracy: 69.4%
Minibatch loss at step 1000: 3.438970
Minibatch accuracy: 80.0%
Validation accuracy: 68.6%
Minibatch loss at step 1500: 2.499999
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Minibatch loss at step 2000: 3.298002
Minibatch accuracy: 80.0%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 2.500329
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 3000: 4.397810
Minibatch accuracy: 60.0%
Validation accuracy: 73.5%
Minibatch loss at step 3500: 11.341330
Minibatch accuracy: 60.0%
Validation accuracy: 75.3%
Minibatch loss at step 4000: 4.994863
Minibatch accuracy: 80.0%
Validation accuracy: 72.0%
Minibatch loss at step 4500: 2.504376
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 5000: 4.524340
Minibatch accuracy: 80.0%
Validation acc

------
Problem 3
------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden1=tf.nn.dropout(hidden1,0.6)

  # Set final layer variables
  weights = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))

  biases = tf.Variable(tf.zeros([num_labels]))
    
  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights) + biases)
  
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [9]:
num_steps = 10001 

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 890.357788
Minibatch accuracy: 10.9%
Validation accuracy: 28.2%
Minibatch loss at step 500: 232.029694
Minibatch accuracy: 78.1%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 105.715851
Minibatch accuracy: 72.7%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 48.840412
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 23.076992
Minibatch accuracy: 89.1%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 11.179365
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 5.648364
Minibatch accuracy: 85.2%
Validation accuracy: 86.0%
Minibatch loss at step 3500: 3.017110
Minibatch accuracy: 82.8%
Validation accuracy: 86.6%
Minibatch loss at step 4000: 1.663760
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Minibatch loss at step 4500: 1.066001
Minibatch accuracy: 89.1%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 0.817069
Minibatch accuracy: 88.3%
Validatio

---
Problem 4
---
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

global_step = tf.Variable(0)  # count the number of steps taken.

learning_rate = tf.train.exponential_decay(0.5, global_step, ...)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

---
